In [53]:
import os
from shutil import copy2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pygrib
from netCDF4 import Dataset

In [2]:
path_to_model     = "/mnt/data-weather/moddata/modgrb/"
path_to_satellite = "/mnt/data-weather/satdata/satnc/band2346/"
path_to_radar     = "/mnt/data-weather/raddata/raddat/"

In [97]:
path_to_model     = "/mnt/data-weather/moddata/modgrb/"
directory = os.fsencode(path_to_model)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print(filename)
    if (len(filename) > 6):
        if (filename[27] == "0" and filename[-1] == "2"):
            print(filename)
            mmdd = filename[15:19]
            hh = filename[20:22]
            rename_in = path_to_model+filename
            rename_out = path_to_model+"band0/nam.2017"+mmdd+"."+hh+".00.grb2"
            print(rename_out)
            os.rename(rename_in, rename_out)
        if (filename[27] == "0" and filename[-1] == "b"):
            print(filename)
            mmdd = filename[15:19]
            hh = filename[20:22]
            rename_in = path_to_model+filename
            rename_out = path_to_model+"band0/nam.2017"+mmdd+"."+hh+".00.grb"
            print(rename_out)
            os.rename(rename_in, rename_out)
        if (filename[27] == "1" and filename[-1] == "2"):
            print(filename)
            mmdd = filename[15:19]
            hh = filename[20:22]
            rename_in = path_to_model+filename
            rename_out = path_to_model+"band1/nam.2017"+mmdd+"."+hh+".01.grb2"
            print(rename_out)
            os.rename(rename_in, rename_out)
        if (filename[27] == "1" and filename[-1] == "b"):
            print(filename)
            mmdd = filename[15:19]
            hh = filename[20:22]
            rename_in = path_to_model+filename
            rename_out = path_to_model+"band1/nam.2017"+mmdd+"."+hh+".01.grb"
            print(rename_out)
            os.rename(rename_in, rename_out)
        if (filename[27] == "2" and filename[-1] == "2"):
            print(filename)
            mmdd = filename[15:19]
            hh = filename[20:22]
            rename_in = path_to_model+filename
            rename_out = path_to_model+"band2/nam.2017"+mmdd+"."+hh+".02.grb2"
            print(rename_out)
            os.rename(rename_in, rename_out)
        if (filename[27] == "2" and filename[-1] == "b"):
            print(filename)
            mmdd = filename[15:19]
            hh = filename[20:22]
            rename_in = path_to_model+filename
            rename_out = path_to_model+"band2/nam.2017"+mmdd+"."+hh+".02.grb"
            print(rename_out)
            os.rename(rename_in, rename_out)
        if (filename[27] == "3" and filename[-1] == "2"):
            print(filename)
            mmdd = filename[15:19]
            hh = filename[20:22]
            rename_in = path_to_model+filename
            rename_out = path_to_model+"band3/nam.2017"+mmdd+"."+hh+".03.grb2"
            print(rename_out)
            os.rename(rename_in, rename_out)
        if (filename[27] == "3" and filename[-1] == "b"):
            print(filename)
            mmdd = filename[15:19]
            hh = filename[20:22]
            rename_in = path_to_model+filename
            rename_out = path_to_model+"band3/nam.2017"+mmdd+"."+hh+".03.grb"
            print(rename_out)
            os.rename(rename_in, rename_out)
        if (filename[27] == "6" and filename[-1] == "2"):
            print(filename)
            mmdd = filename[15:19]
            hh = filename[20:22]
            rename_in = path_to_model+filename
            rename_out = path_to_model+"band6/nam.2017"+mmdd+"."+hh+".06.grb2"
            print(rename_out)
            os.rename(rename_in, rename_out)
        if (filename[27] == "6" and filename[-1] == "b"):
            print(filename)
            mmdd = filename[15:19]
            hh = filename[20:22]
            rename_in = path_to_model+filename
            rename_out = path_to_model+"band6/nam.2017"+mmdd+"."+hh+".06.grb"
            print(rename_out)
            os.rename(rename_in, rename_out)

band0
band1
band2
band3
band6
namanl_218_20170101_0000_000.grb
namanl_218_20170101_0000_000.grb
/mnt/data-weather/moddata/modgrb/band0/nam.20170101.00.00.grb
namanl_218_20170101_0000_001.grb
namanl_218_20170101_0000_001.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170101.00.01.grb
namanl_218_20170101_0000_002.grb
namanl_218_20170101_0000_002.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170101.00.02.grb
namanl_218_20170101_0000_003.grb
namanl_218_20170101_0000_003.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170101.00.03.grb
namanl_218_20170101_0000_006.grb
namanl_218_20170101_0000_006.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170101.00.06.grb
namanl_218_20170101_0600_000.grb
namanl_218_20170101_0600_000.grb
/mnt/data-weather/moddata/modgrb/band0/nam.20170101.06.00.grb
namanl_218_20170101_0600_001.grb
namanl_218_20170101_0600_001.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170101.06.01.grb
namanl_218_20170101_0600_002.grb
namanl_218_20170101_0600_002.grb
/mnt/dat

namanl_218_20170105_0000_006.grb
namanl_218_20170105_0000_006.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170105.00.06.grb
namanl_218_20170105_0600_000.grb
namanl_218_20170105_0600_000.grb
/mnt/data-weather/moddata/modgrb/band0/nam.20170105.06.00.grb
namanl_218_20170105_0600_001.grb
namanl_218_20170105_0600_001.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170105.06.01.grb
namanl_218_20170105_0600_002.grb
namanl_218_20170105_0600_002.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170105.06.02.grb
namanl_218_20170105_0600_003.grb
namanl_218_20170105_0600_003.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170105.06.03.grb
namanl_218_20170105_0600_006.grb
namanl_218_20170105_0600_006.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170105.06.06.grb
namanl_218_20170105_1200_000.grb
namanl_218_20170105_1200_000.grb
/mnt/data-weather/moddata/modgrb/band0/nam.20170105.12.00.grb
namanl_218_20170105_1200_001.grb
namanl_218_20170105_1200_001.grb
/mnt/data-weather/moddata/modgrb/band1

In [111]:
path_to_model = "/mnt/data-weather/moddata/modgrb/band1/"
directory = os.fsencode(path_to_model)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if (filename[-1] == "2"):
        rename_in = path_to_model + filename
        #print(rename_in)
        rename_out = path_to_model + filename[0:17] + "1.grb2"
        print(rename_out)
        os.rename(rename_in, rename_out)
    if (filename[-1] == "b"):
        rename_in = path_to_model + filename
        #print(rename_in)
        rename_out = path_to_model + filename[0:17] + "1.grb"
        print(rename_out)
        os.rename(rename_in, rename_out)

/mnt/data-weather/moddata/modgrb/band1/nam.20171226.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171226.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171226.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171227.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171227.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171227.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171227.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171228.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171228.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171228.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171228.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171229.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171229.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171229.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171229.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171230.00.

/mnt/data-weather/moddata/modgrb/band1/nam.20171122.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171122.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171123.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171123.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171123.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171123.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171124.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171124.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171124.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171124.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171125.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171125.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171125.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171125.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171126.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20171126.06.

/mnt/data-weather/moddata/modgrb/band1/nam.20170913.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170914.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170914.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170914.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170914.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170915.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170915.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170915.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170915.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170916.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170916.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170916.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170916.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170917.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170917.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170917.12.

/mnt/data-weather/moddata/modgrb/band1/nam.20170902.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170902.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170903.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170903.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170903.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170903.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170904.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170904.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170904.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170904.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170905.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170905.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170905.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170905.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170906.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170906.06.

/mnt/data-weather/moddata/modgrb/band1/nam.20170630.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170630.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170701.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170701.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170701.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170701.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170702.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170702.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170702.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170702.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170703.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170703.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170703.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170703.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170704.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170704.06.

/mnt/data-weather/moddata/modgrb/band1/nam.20170428.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170428.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170428.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170428.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170429.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170429.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170429.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170429.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170430.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170430.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170430.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170430.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170501.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170501.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170501.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170501.18.

/mnt/data-weather/moddata/modgrb/band1/nam.20170419.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170419.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170419.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170419.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170420.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170420.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170420.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170420.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170421.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170421.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170421.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170421.18.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170422.00.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170422.06.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170422.12.01.grb2
/mnt/data-weather/moddata/modgrb/band1/nam.20170422.18.

/mnt/data-weather/moddata/modgrb/band1/nam.20170206.12.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170206.18.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170207.00.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170207.06.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170207.12.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170207.18.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170208.00.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170208.06.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170208.12.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170208.18.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170209.00.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170209.06.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170209.12.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170209.18.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170210.00.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170210.06.01.grb
/mnt/dat

/mnt/data-weather/moddata/modgrb/band1/nam.20170127.12.01.grb
/mnt/data-weather/moddata/modgrb/band1/nam.20170127.18.01.grb


In [112]:
path_to_model = "/mnt/data-weather/moddata/modgrb/band2/"
directory = os.fsencode(path_to_model)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if (filename[-1] == "2"):
        rename_in = path_to_model + filename
        #print(rename_in)
        rename_out = path_to_model + filename[0:17] + "2.grb2"
        print(rename_out)
        os.rename(rename_in, rename_out)
    if (filename[-1] == "b"):
        rename_in = path_to_model + filename
        #print(rename_in)
        rename_out = path_to_model + filename[0:17] + "2.grb"
        print(rename_out)
        os.rename(rename_in, rename_out)

/mnt/data-weather/moddata/modgrb/band2/nam.20171226.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171226.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171226.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171227.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171227.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171227.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171227.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171228.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171228.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171228.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171228.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171229.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171229.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171229.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171229.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171230.00.

/mnt/data-weather/moddata/modgrb/band2/nam.20171105.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171105.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171105.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171105.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171106.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171106.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171106.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171106.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171107.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171107.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171107.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171107.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171108.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171108.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171108.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171108.18.

/mnt/data-weather/moddata/modgrb/band2/nam.20171021.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171021.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171022.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171022.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171022.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171022.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171023.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171023.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171023.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171023.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171024.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171024.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171024.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171024.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171025.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20171025.06.

/mnt/data-weather/moddata/modgrb/band2/nam.20170908.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170715.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170716.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170716.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170716.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170716.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170717.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170717.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170717.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170717.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170718.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170718.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170718.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170718.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170719.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170719.06.

/mnt/data-weather/moddata/modgrb/band2/nam.20170605.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170606.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170606.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170606.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170606.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170607.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170607.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170607.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170607.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170608.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170608.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170608.12.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170608.18.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170609.00.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170609.06.02.grb2
/mnt/data-weather/moddata/modgrb/band2/nam.20170609.12.

/mnt/data-weather/moddata/modgrb/band2/nam.20170330.12.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170330.18.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170331.00.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170331.06.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170331.12.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170331.18.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170401.00.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170401.06.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170401.12.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170401.18.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170402.00.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170402.06.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170402.12.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170402.18.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170403.00.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170403.06.02.grb
/mnt/dat

/mnt/data-weather/moddata/modgrb/band2/nam.20170311.00.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170311.06.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170311.12.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170311.18.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170312.00.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170312.06.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170312.12.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170312.18.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170313.00.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170313.06.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170313.12.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170313.18.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170314.00.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170314.06.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170314.12.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170314.18.02.grb
/mnt/dat

/mnt/data-weather/moddata/modgrb/band2/nam.20170110.18.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170111.00.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170111.06.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170111.12.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170111.18.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170112.00.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170112.06.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170112.12.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170112.18.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170113.00.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170113.06.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170113.12.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170113.18.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170114.00.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170114.06.02.grb
/mnt/data-weather/moddata/modgrb/band2/nam.20170114.12.02.grb
/mnt/dat

In [113]:
path_to_model = "/mnt/data-weather/moddata/modgrb/band3/"
directory = os.fsencode(path_to_model)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if (filename[-1] == "2"):
        rename_in = path_to_model + filename
        #print(rename_in)
        rename_out = path_to_model + filename[0:17] + "3.grb2"
        print(rename_out)
        os.rename(rename_in, rename_out)
    if (filename[-1] == "b"):
        rename_in = path_to_model + filename
        #print(rename_in)
        rename_out = path_to_model + filename[0:17] + "3.grb"
        print(rename_out)
        os.rename(rename_in, rename_out)

/mnt/data-weather/moddata/modgrb/band3/nam.20171226.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171226.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171226.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171227.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171227.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171227.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171227.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171228.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171228.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171228.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171228.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171229.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171229.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171229.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171229.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171230.00.

/mnt/data-weather/moddata/modgrb/band3/nam.20171120.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171121.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171121.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171121.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171121.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171122.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171122.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171122.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171122.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171123.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171123.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171123.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171123.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171124.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171124.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20171124.12.

/mnt/data-weather/moddata/modgrb/band3/nam.20170913.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170913.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170913.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170913.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170914.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170914.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170914.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170914.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170915.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170915.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170915.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170915.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170916.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170916.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170916.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170916.18.

/mnt/data-weather/moddata/modgrb/band3/nam.20170901.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170901.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170902.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170902.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170902.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170902.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170903.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170903.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170903.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170903.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170904.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170904.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170904.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170904.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170905.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170905.06.

/mnt/data-weather/moddata/modgrb/band3/nam.20170627.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170627.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170627.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170627.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170628.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170628.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170628.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170628.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170629.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170629.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170629.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170629.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170630.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170630.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170630.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170630.18.

/mnt/data-weather/moddata/modgrb/band3/nam.20170608.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170608.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170608.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170609.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170609.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170609.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170609.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170610.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170610.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170610.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170610.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170611.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170611.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170611.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170611.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170612.00.

/mnt/data-weather/moddata/modgrb/band3/nam.20170415.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170415.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170416.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170416.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170416.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170416.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170417.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170417.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170417.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170417.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170418.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170418.06.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170418.12.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170418.18.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170419.00.03.grb2
/mnt/data-weather/moddata/modgrb/band3/nam.20170419.06.

/mnt/data-weather/moddata/modgrb/band3/nam.20170130.00.03.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170130.06.03.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170130.12.03.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170130.18.03.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170131.00.03.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170131.06.03.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170131.12.03.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170131.18.03.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170201.00.03.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170201.06.03.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170201.12.03.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170201.18.03.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170202.00.03.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170202.06.03.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170202.12.03.grb
/mnt/data-weather/moddata/modgrb/band3/nam.20170202.18.03.grb
/mnt/dat

In [114]:
path_to_model = "/mnt/data-weather/moddata/modgrb/band6/"
directory = os.fsencode(path_to_model)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if (filename[-1] == "2"):
        rename_in = path_to_model + filename
        #print(rename_in)
        rename_out = path_to_model + filename[0:17] + "6.grb2"
        print(rename_out)
        os.rename(rename_in, rename_out)
    if (filename[-1] == "b"):
        rename_in = path_to_model + filename
        #print(rename_in)
        rename_out = path_to_model + filename[0:17] + "6.grb"
        print(rename_out)
        os.rename(rename_in, rename_out)

/mnt/data-weather/moddata/modgrb/band6/nam.20171226.06.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171226.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171226.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171227.00.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171227.06.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171227.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171227.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171228.00.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171228.06.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171228.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171228.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171229.00.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171229.06.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171229.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171229.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171230.00.

/mnt/data-weather/moddata/modgrb/band6/nam.20171128.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171128.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171005.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171005.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171006.00.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171006.06.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171006.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171006.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171007.00.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171007.06.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171007.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171007.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171008.00.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171008.06.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171008.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20171008.18.

/mnt/data-weather/moddata/modgrb/band6/nam.20170907.06.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170907.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170907.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170908.00.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170715.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170716.00.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170716.06.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170716.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170716.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170717.00.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170717.06.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170717.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170717.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170718.00.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170718.06.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170718.12.

/mnt/data-weather/moddata/modgrb/band6/nam.20170505.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170505.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170506.00.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170506.06.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170506.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170506.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170507.00.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170507.06.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170507.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170507.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170508.00.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170508.06.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170508.12.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170508.18.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170509.00.06.grb2
/mnt/data-weather/moddata/modgrb/band6/nam.20170509.06.

/mnt/data-weather/moddata/modgrb/band6/nam.20170221.18.06.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170222.00.06.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170222.06.06.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170222.12.06.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170222.18.06.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170223.00.06.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170223.06.06.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170223.12.06.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170223.18.06.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170224.00.06.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170224.06.06.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170101.00.06.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170101.06.06.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170101.12.06.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170101.18.06.grb
/mnt/data-weather/moddata/modgrb/band6/nam.20170102.00.06.grb
/mnt/dat

In [ ]:
path_to_model = "/mnt/data-weather/moddata/modgrb/band3/"
directory = os.fsencode(path_to_model)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if (filename[-1] == "2"):
        rename_in = path_to_model + filename
        #print(rename_in)
        rename_out = path_to_model + filename[0:17] + "3.grb2"
        print(rename_out)
        os.rename(rename_in, rename_out)
    if (filename[-1] == "b"):
        rename_in = path_to_model + filename
        #print(rename_in)
        rename_out = path_to_model + filename[0:17] + "3.grb"
        print(rename_out)
        os.rename(rename_in, rename_out)

In [ ]:
path_to_model_output     = "/mnt/data-weather/moddata/modpng/"
path_to_satellite_output = "/mnt/data-weather/satdata/satpng/band2346/"
path_to_radar_output     = "/mnt/data-weather/raddata/radpng/"

In [65]:
path_to_model        = "/mnt/data-weather/moddata/modgrb/"
path_to_model_output = "/mnt/data-weather/modpng/"
path_to_radar        = "/mnt/data-weather/raddata/raddat"
a = "/mnt/data-weather/a/"
directory1 = os.fsencode(path_to_model)
directory2 = os.fsencode(path_to_radar)

for file1 in os.listdir(directory1):
    filename1 = os.fsdecode(file1)
    x = int(filename1[20:22])
    y = int(filename1[27])
    if (y != 6 and filename1[-1] != "2" and x <=7):
        print(filename1)
        out = a+"mod.2017"+filename1[15:19]+"0"+str(x+y)+".grb"
        print(out)
        copy2(path_to_model+filename1,out)
    if (y != 6 and filename1[-1] != "2" and x >7):
        print(filename1)
        out = a+"mod.2017"+filename1[15:19]+str(x+y)+".grb"
        print(out)
        copy2(path_to_model+filename1,out)
    if (y != 6 and filename1[-1] == "2" and x <=7):
        print(filename1)
        out = a+"mod.2017"+filename1[15:19]+"0"+str(x+y)+".grb2"
        print(out)
        copy2(path_to_model+filename1,out)
    if (y != 6 and filename1[-1] == "2" and x >7):
        print(filename1)
        out = a+"mod.2017"+filename1[15:19]+str(x+y)+".grb2"
        print(out)
        copy2(path_to_model+filename1,out)

namanl_218_20171230_0000_001.grb2
/mnt/data-weather/a/mod.2017123001.grb2
namanl_218_20171230_0000_002.grb2
/mnt/data-weather/a/mod.2017123002.grb2
namanl_218_20171230_0000_003.grb2
/mnt/data-weather/a/mod.2017123003.grb2
namanl_218_20171230_0600_000.grb2
/mnt/data-weather/a/mod.2017123006.grb2
namanl_218_20171230_0600_001.grb2
/mnt/data-weather/a/mod.2017123007.grb2
namanl_218_20171230_0600_002.grb2
/mnt/data-weather/a/mod.2017123008.grb2
namanl_218_20171230_0600_003.grb2
/mnt/data-weather/a/mod.2017123009.grb2
namanl_218_20171230_1200_000.grb2
/mnt/data-weather/a/mod.2017123012.grb2
namanl_218_20171230_1200_001.grb2
/mnt/data-weather/a/mod.2017123013.grb2
namanl_218_20171230_1200_002.grb2
/mnt/data-weather/a/mod.2017123014.grb2
namanl_218_20171230_1200_003.grb2
/mnt/data-weather/a/mod.2017123015.grb2
namanl_218_20171230_1800_000.grb2
/mnt/data-weather/a/mod.2017123018.grb2
namanl_218_20171230_1800_001.grb2
/mnt/data-weather/a/mod.2017123019.grb2
namanl_218_20171230_1800_002.grb2
/mnt

namanl_218_20171230_0000_000.grb2
/mnt/data-weather/a/mod.2017123000.grb2
namanl_218_20171219_1800_002.grb2
/mnt/data-weather/a/mod.2017121920.grb2
namanl_218_20171219_1800_003.grb2
/mnt/data-weather/a/mod.2017121921.grb2
namanl_218_20171220_0000_000.grb2
/mnt/data-weather/a/mod.2017122000.grb2
namanl_218_20171220_0000_001.grb2
/mnt/data-weather/a/mod.2017122001.grb2
namanl_218_20171220_0000_002.grb2
/mnt/data-weather/a/mod.2017122002.grb2
namanl_218_20171220_0000_003.grb2
/mnt/data-weather/a/mod.2017122003.grb2
namanl_218_20171220_0600_000.grb2
/mnt/data-weather/a/mod.2017122006.grb2
namanl_218_20171220_0600_001.grb2
/mnt/data-weather/a/mod.2017122007.grb2
namanl_218_20171220_0600_002.grb2
/mnt/data-weather/a/mod.2017122008.grb2
namanl_218_20171220_0600_003.grb2
/mnt/data-weather/a/mod.2017122009.grb2
namanl_218_20171220_1200_000.grb2
/mnt/data-weather/a/mod.2017122012.grb2
namanl_218_20171220_1200_001.grb2
/mnt/data-weather/a/mod.2017122013.grb2
namanl_218_20171220_1200_002.grb2
/mnt

namanl_218_20171216_1200_000.grb2
/mnt/data-weather/a/mod.2017121612.grb2
namanl_218_20171216_1200_001.grb2
/mnt/data-weather/a/mod.2017121613.grb2
namanl_218_20171216_1200_002.grb2
/mnt/data-weather/a/mod.2017121614.grb2
namanl_218_20171216_1200_003.grb2
/mnt/data-weather/a/mod.2017121615.grb2
namanl_218_20171216_1800_000.grb2
/mnt/data-weather/a/mod.2017121618.grb2
namanl_218_20171216_1800_001.grb2
/mnt/data-weather/a/mod.2017121619.grb2
namanl_218_20171216_1800_002.grb2
/mnt/data-weather/a/mod.2017121620.grb2
namanl_218_20171216_1800_003.grb2
/mnt/data-weather/a/mod.2017121621.grb2
namanl_218_20171217_0000_000.grb2
/mnt/data-weather/a/mod.2017121700.grb2
namanl_218_20171217_0000_001.grb2
/mnt/data-weather/a/mod.2017121701.grb2
namanl_218_20171217_0000_002.grb2
/mnt/data-weather/a/mod.2017121702.grb2
namanl_218_20171217_0000_003.grb2
/mnt/data-weather/a/mod.2017121703.grb2
namanl_218_20171217_0600_000.grb2
/mnt/data-weather/a/mod.2017121706.grb2
namanl_218_20171217_0600_001.grb2
/mnt

namanl_218_20171213_0600_000.grb2
/mnt/data-weather/a/mod.2017121306.grb2
namanl_218_20171213_0600_001.grb2
/mnt/data-weather/a/mod.2017121307.grb2
namanl_218_20171213_0600_002.grb2
/mnt/data-weather/a/mod.2017121308.grb2
namanl_218_20171213_0600_003.grb2
/mnt/data-weather/a/mod.2017121309.grb2
namanl_218_20171213_1200_000.grb2
/mnt/data-weather/a/mod.2017121312.grb2
namanl_218_20171213_1200_001.grb2
/mnt/data-weather/a/mod.2017121313.grb2
namanl_218_20171213_1200_002.grb2
/mnt/data-weather/a/mod.2017121314.grb2
namanl_218_20171213_1200_003.grb2
/mnt/data-weather/a/mod.2017121315.grb2
namanl_218_20171213_1800_000.grb2
/mnt/data-weather/a/mod.2017121318.grb2
namanl_218_20171213_1800_001.grb2
/mnt/data-weather/a/mod.2017121319.grb2
namanl_218_20171213_1800_002.grb2
/mnt/data-weather/a/mod.2017121320.grb2
namanl_218_20171213_1800_003.grb2
/mnt/data-weather/a/mod.2017121321.grb2
namanl_218_20171214_0000_000.grb2
/mnt/data-weather/a/mod.2017121400.grb2
namanl_218_20171214_0000_001.grb2
/mnt

namanl_218_20171129_1800_001.grb2
/mnt/data-weather/a/mod.2017112919.grb2
namanl_218_20171129_1800_002.grb2
/mnt/data-weather/a/mod.2017112920.grb2
namanl_218_20171129_1800_003.grb2
/mnt/data-weather/a/mod.2017112921.grb2
namanl_218_20171130_0000_000.grb2
/mnt/data-weather/a/mod.2017113000.grb2
namanl_218_20171130_0000_001.grb2
/mnt/data-weather/a/mod.2017113001.grb2
namanl_218_20171130_0000_002.grb2
/mnt/data-weather/a/mod.2017113002.grb2
namanl_218_20171130_0000_003.grb2
/mnt/data-weather/a/mod.2017113003.grb2
namanl_218_20171130_0600_000.grb2
/mnt/data-weather/a/mod.2017113006.grb2
namanl_218_20171130_0600_001.grb2
/mnt/data-weather/a/mod.2017113007.grb2
namanl_218_20171130_0600_002.grb2
/mnt/data-weather/a/mod.2017113008.grb2
namanl_218_20171130_0600_003.grb2
/mnt/data-weather/a/mod.2017113009.grb2
namanl_218_20171130_1200_000.grb2
/mnt/data-weather/a/mod.2017113012.grb2
namanl_218_20171130_1200_001.grb2
/mnt/data-weather/a/mod.2017113013.grb2
namanl_218_20171130_1200_002.grb2
/mnt

namanl_218_20171126_1200_001.grb2
/mnt/data-weather/a/mod.2017112613.grb2
namanl_218_20171126_1200_002.grb2
/mnt/data-weather/a/mod.2017112614.grb2
namanl_218_20171126_1200_003.grb2
/mnt/data-weather/a/mod.2017112615.grb2
namanl_218_20171126_1800_000.grb2
/mnt/data-weather/a/mod.2017112618.grb2
namanl_218_20171126_1800_001.grb2
/mnt/data-weather/a/mod.2017112619.grb2
namanl_218_20171126_1800_002.grb2
/mnt/data-weather/a/mod.2017112620.grb2
namanl_218_20171126_1800_003.grb2
/mnt/data-weather/a/mod.2017112621.grb2
namanl_218_20171127_0000_000.grb2
/mnt/data-weather/a/mod.2017112700.grb2
namanl_218_20171127_0000_001.grb2
/mnt/data-weather/a/mod.2017112701.grb2
namanl_218_20171127_0000_002.grb2
/mnt/data-weather/a/mod.2017112702.grb2
namanl_218_20171127_0000_003.grb2
/mnt/data-weather/a/mod.2017112703.grb2
namanl_218_20171127_0600_000.grb2
/mnt/data-weather/a/mod.2017112706.grb2
namanl_218_20171127_0600_001.grb2
/mnt/data-weather/a/mod.2017112707.grb2
namanl_218_20171127_0600_002.grb2
/mnt

KeyboardInterrupt: 

In [2]:
path_to_satellite = "/mnt/data-weather/corr/sat/band3/"
path_to_satellite_output = "/mnt/data-weather/corr/A3/"
directory = os.fsencode(path_to_satellite)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    #print(filename)
    nc = Dataset(path_to_satellite+filename, 'r')
    #extracting the data we want from the file
    data = nc.variables['data'][:][0]
    lat = nc.variables['lat'][:]
    lon = nc.variables['lon'][:]
    nc.close()
    #print(data.shape, lat.shape, lon.shape)
    plt.figure(figsize=(10,10))
    m = Basemap(projection='mill',
            resolution=None,
            llcrnrlon=-102,
            urcrnrlon=-82,
            llcrnrlat=31,
            urcrnrlat=41)
    x,y = m(lon,lat)
    cs = m.pcolormesh(x,y,data,shading='flat',cmap="gray")
    plt.savefig(path_to_satellite_output+filename[0:-3]+".png",
                bbox_inches='tight',
                pad_inches=-0.08)
    plt.clf()
    plt.close()

In [ ]:
import os
from shutil import copy2
path_to_satellite = "/mnt/data-weather/satdata/satnc/band2346/"
path_to_other = "/mnt/data-weather/satdata/satnc/other/"
directory = os.fsencode(path_to_satellite)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    #print(filename[18:20])
    if filename[18:20] == "15":
        copy2(path_to_satellite+filename, path_to_other+filename)

In [ ]:
from datetime import timedelta
import datetime
directory = os.fsencode(path_to_other)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print(filename)
    print(filename[12:15])
    date = datetime.datetime(2017, 1, 1) + datetime.timedelta(int(filename[12:15]) - 1)
    if (date.month <= 9 and date.day <= 9):
        #print("sat."+str(date.year)+"0"+str(date.month)+"0"+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc")
        name = "sat."+str(date.year)+"0"+str(date.month)+"0"+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc"
    elif (date.month <= 9 and date.day > 9):
        #print("sat."+str(date.year)+"0"+str(date.month)+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc")
        name = "sat."+str(date.year)+"0"+str(date.month)+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc"
    elif (date.month > 9 and date.day <= 9):
        #print("sat."+str(date.year)+str(date.month)+"0"+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc")
        name = "sat."+str(date.year)+str(date.month)+"0"+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc"
    else:
        #print("sat."+str(date.year)+str(date.month)+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc")
        name = "sat."+str(date.year)+str(date.month)+str(date.day)+"."+filename[16:18]+"."+filename[28:30]+".nc"
    os.rename(path_to_other+filename, path_to_other+name)

In [ ]:
sat = "/mnt/data-weather/satdata/satnc/other/"
rad = "/mnt/data-weather/raddata/raddat/"
sat_out = "/mnt/data-weather/corr/sat/"
rad_out = "/mnt/data-weather/corr/rad/"
sat_dir = os.fsencode(sat)
rad_dir = os.fsencode(rad)

for sat_file in os.listdir(sat_dir):
    sat_filename = os.fsdecode(sat_file)
    for rad_file in os.listdir(rad_dir):
        rad_filename = os.fsdecode(rad_file)
        #print(sat_filename[8:15], rad_filename[10:17])
        if sat_filename[8:15] == rad_filename[10:17]:
            copy2(sat+sat_filename, sat_out+sat_filename)
            copy2(rad+rad_filename, rad_out+rad_filename)

In [ ]:
sat = "/mnt/data-weather/corr/sat/"
sat_out = "/mnt/data-weather/corr/sat/band"
sat_dir = os.fsencode(sat)
for sat_file in os.listdir(sat_dir):
    file = os.fsdecode(sat_file)
    print(file)
    print(file[17:18])
    if len(file) > 5:
        if int(file[17:18]) == 2:
            os.rename(sat+file,sat_out+"2/"+file)
        if int(file[17:18]) == 3:
            os.rename(sat+file,sat_out+"3/"+file)   
        if int(file[17:18]) == 4:
            os.rename(sat+file,sat_out+"4/"+file)
        if int(file[17:18]) == 6:
            os.rename(sat+file,sat_out+"6/"+file)

In [ ]:
rad = "/mnt/data-weather/corr/rad/"
out = "/mnt/data-weather/corr/B2/"
directory = os.fsencode(rad)
dims = (126,201)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    #print(filename)
    data = np.fromfile(rad+filename,dtype='int16',count=-1,sep='') # 2 byte integers
    #print('size of read vector',len(data))
    data = np.reshape(data,dims)
    #print(data)
    fig = plt.figure(figsize=(10,10))
    plt.axis('off')
    plt.imshow(data, cmap='gray', origin='lower')
    fig.savefig(out+filename[0:-4]+".png",
                bbox_inches='tight',
                pad_inches=-0.36)
    plt.close(fig)

In [2]:
from shutil import copy2

In [15]:
rad = "/mnt/data-weather/corr/B2/"
rad_out = "/mnt/data-weather/corr/B/"
sat = "/mnt/data-weather/corr/A2/"
sat_out = "/mnt/data-weather/corr/A/"
rad_dir = os.fsencode(rad)
sat_dir = os.fsencode(sat)
#for file in os.listdir(rad_dir):
    #filename = os.fsdecode(file)
    #print(rad_out+filename[10:14]+filename[15:17]+"B2")
    #copy2(rad+filename,rad_out+filename[10:14]+filename[15:17]+"B2.png")
    
#for file in os.listdir(sat_dir):
    #filename = os.fsdecode(file)
    #print(sat_out+filename[8:12]+filename[13:15]+"A2")
    #copy2(sat+filename,sat_out+filename[8:12]+filename[13:15]+"A2.png")

/mnt/data-weather/corr/A/102903A2
/mnt/data-weather/corr/A/102904A2
/mnt/data-weather/corr/A/102905A2
/mnt/data-weather/corr/A/102906A2
/mnt/data-weather/corr/A/102907A2
/mnt/data-weather/corr/A/102908A2
/mnt/data-weather/corr/A/102909A2
/mnt/data-weather/corr/A/110519A2
/mnt/data-weather/corr/A/110520A2
/mnt/data-weather/corr/A/110521A2
/mnt/data-weather/corr/A/110522A2
/mnt/data-weather/corr/A/110523A2
/mnt/data-weather/corr/A/110600A2
/mnt/data-weather/corr/A/110601A2
/mnt/data-weather/corr/A/110602A2
/mnt/data-weather/corr/A/110603A2
/mnt/data-weather/corr/A/110604A2
/mnt/data-weather/corr/A/110605A2
/mnt/data-weather/corr/A/120500A2
/mnt/data-weather/corr/A/120501A2
/mnt/data-weather/corr/A/120502A2
/mnt/data-weather/corr/A/120503A2
/mnt/data-weather/corr/A/120504A2
/mnt/data-weather/corr/A/120505A2
/mnt/data-weather/corr/A/120817A2
/mnt/data-weather/corr/A/120818A2
/mnt/data-weather/corr/A/120819A2
/mnt/data-weather/corr/A/120820A2
/mnt/data-weather/corr/A/120821A2
/mnt/data-weat

/mnt/data-weather/corr/A/081702A2
/mnt/data-weather/corr/A/081703A2
/mnt/data-weather/corr/A/081817A2
/mnt/data-weather/corr/A/081818A2
/mnt/data-weather/corr/A/081819A2
/mnt/data-weather/corr/A/081820A2
/mnt/data-weather/corr/A/081821A2
/mnt/data-weather/corr/A/081822A2
/mnt/data-weather/corr/A/081823A2
/mnt/data-weather/corr/A/081900A2
/mnt/data-weather/corr/A/081901A2
/mnt/data-weather/corr/A/081902A2
/mnt/data-weather/corr/A/081903A2
/mnt/data-weather/corr/A/082216A2
/mnt/data-weather/corr/A/082217A2
/mnt/data-weather/corr/A/082218A2
/mnt/data-weather/corr/A/082219A2
/mnt/data-weather/corr/A/082220A2
/mnt/data-weather/corr/A/082221A2
/mnt/data-weather/corr/A/082222A2
/mnt/data-weather/corr/A/082223A2
/mnt/data-weather/corr/A/082300A2
/mnt/data-weather/corr/A/082301A2
/mnt/data-weather/corr/A/082314A2
/mnt/data-weather/corr/A/082315A2
/mnt/data-weather/corr/A/082316A2
/mnt/data-weather/corr/A/082317A2
/mnt/data-weather/corr/A/082318A2
/mnt/data-weather/corr/A/082319A2
/mnt/data-weat

/mnt/data-weather/corr/A/070706A2
/mnt/data-weather/corr/A/070716A2
/mnt/data-weather/corr/A/070717A2
/mnt/data-weather/corr/A/070718A2
/mnt/data-weather/corr/A/070719A2
/mnt/data-weather/corr/A/070720A2
/mnt/data-weather/corr/A/070721A2
/mnt/data-weather/corr/A/070722A2
/mnt/data-weather/corr/A/070723A2
/mnt/data-weather/corr/A/070800A2
/mnt/data-weather/corr/A/070801A2
/mnt/data-weather/corr/A/070802A2
/mnt/data-weather/corr/A/070803A2
/mnt/data-weather/corr/A/070919A2
/mnt/data-weather/corr/A/070920A2
/mnt/data-weather/corr/A/070921A2
/mnt/data-weather/corr/A/070922A2
/mnt/data-weather/corr/A/070923A2
/mnt/data-weather/corr/A/071000A2
/mnt/data-weather/corr/A/071001A2
/mnt/data-weather/corr/A/071002A2
/mnt/data-weather/corr/A/071003A2
/mnt/data-weather/corr/A/071004A2
/mnt/data-weather/corr/A/071005A2
/mnt/data-weather/corr/A/071006A2
/mnt/data-weather/corr/A/071007A2
/mnt/data-weather/corr/A/071008A2
/mnt/data-weather/corr/A/071009A2
/mnt/data-weather/corr/A/071010A2
/mnt/data-weat

/mnt/data-weather/corr/A/062112A2
/mnt/data-weather/corr/A/062113A2
/mnt/data-weather/corr/A/062114A2
/mnt/data-weather/corr/A/062115A2
/mnt/data-weather/corr/A/062116A2
/mnt/data-weather/corr/A/062117A2
/mnt/data-weather/corr/A/062118A2
/mnt/data-weather/corr/A/062119A2
/mnt/data-weather/corr/A/062120A2
/mnt/data-weather/corr/A/062121A2
/mnt/data-weather/corr/A/062122A2
/mnt/data-weather/corr/A/062123A2
/mnt/data-weather/corr/A/062200A2
/mnt/data-weather/corr/A/062201A2
/mnt/data-weather/corr/A/062202A2
/mnt/data-weather/corr/A/062203A2
/mnt/data-weather/corr/A/062204A2
/mnt/data-weather/corr/A/062205A2
/mnt/data-weather/corr/A/062206A2
/mnt/data-weather/corr/A/062207A2
/mnt/data-weather/corr/A/062208A2
/mnt/data-weather/corr/A/062209A2
/mnt/data-weather/corr/A/052622A2
/mnt/data-weather/corr/A/052623A2
/mnt/data-weather/corr/A/052700A2
/mnt/data-weather/corr/A/052701A2
/mnt/data-weather/corr/A/052702A2
/mnt/data-weather/corr/A/052703A2
/mnt/data-weather/corr/A/052704A2
/mnt/data-weat

/mnt/data-weather/corr/A/050121A2
/mnt/data-weather/corr/A/050122A2
/mnt/data-weather/corr/A/050123A2
/mnt/data-weather/corr/A/050200A2
/mnt/data-weather/corr/A/050201A2
/mnt/data-weather/corr/A/050202A2
/mnt/data-weather/corr/A/050203A2
/mnt/data-weather/corr/A/050204A2
/mnt/data-weather/corr/A/050205A2
/mnt/data-weather/corr/A/050312A2
/mnt/data-weather/corr/A/050313A2
/mnt/data-weather/corr/A/050314A2
/mnt/data-weather/corr/A/050315A2
/mnt/data-weather/corr/A/050316A2
/mnt/data-weather/corr/A/050317A2
/mnt/data-weather/corr/A/050318A2
/mnt/data-weather/corr/A/050319A2
/mnt/data-weather/corr/A/050320A2
/mnt/data-weather/corr/A/050321A2
/mnt/data-weather/corr/A/050322A2
/mnt/data-weather/corr/A/050323A2
/mnt/data-weather/corr/A/050400A2
/mnt/data-weather/corr/A/050401A2
/mnt/data-weather/corr/A/050402A2
/mnt/data-weather/corr/A/050403A2
/mnt/data-weather/corr/A/050404A2
/mnt/data-weather/corr/A/050405A2
/mnt/data-weather/corr/A/050406A2
/mnt/data-weather/corr/A/050407A2
/mnt/data-weat

/mnt/data-weather/corr/A/041711A2
/mnt/data-weather/corr/A/041712A2
/mnt/data-weather/corr/A/041719A2
/mnt/data-weather/corr/A/041720A2
/mnt/data-weather/corr/A/041721A2
/mnt/data-weather/corr/A/041722A2
/mnt/data-weather/corr/A/041723A2
/mnt/data-weather/corr/A/041800A2
/mnt/data-weather/corr/A/041801A2
/mnt/data-weather/corr/A/041802A2
/mnt/data-weather/corr/A/041803A2
/mnt/data-weather/corr/A/041804A2
/mnt/data-weather/corr/A/041805A2
/mnt/data-weather/corr/A/041914A2
/mnt/data-weather/corr/A/041915A2
/mnt/data-weather/corr/A/041916A2
/mnt/data-weather/corr/A/041917A2
/mnt/data-weather/corr/A/041918A2
/mnt/data-weather/corr/A/041919A2
/mnt/data-weather/corr/A/041920A2
/mnt/data-weather/corr/A/041921A2
/mnt/data-weather/corr/A/041922A2
/mnt/data-weather/corr/A/042000A2
/mnt/data-weather/corr/A/042001A2
/mnt/data-weather/corr/A/042002A2
/mnt/data-weather/corr/A/042003A2
/mnt/data-weather/corr/A/042004A2
/mnt/data-weather/corr/A/042005A2
/mnt/data-weather/corr/A/042006A2
/mnt/data-weat

/mnt/data-weather/corr/A/032504A2
/mnt/data-weather/corr/A/032505A2
/mnt/data-weather/corr/A/032506A2
/mnt/data-weather/corr/A/032619A2
/mnt/data-weather/corr/A/032620A2
/mnt/data-weather/corr/A/032621A2
/mnt/data-weather/corr/A/032622A2
/mnt/data-weather/corr/A/032623A2
/mnt/data-weather/corr/A/032700A2
/mnt/data-weather/corr/A/032701A2
/mnt/data-weather/corr/A/032702A2
/mnt/data-weather/corr/A/032703A2
/mnt/data-weather/corr/A/032716A2
/mnt/data-weather/corr/A/032717A2
/mnt/data-weather/corr/A/032718A2
/mnt/data-weather/corr/A/032719A2
/mnt/data-weather/corr/A/032720A2
/mnt/data-weather/corr/A/032721A2
/mnt/data-weather/corr/A/032722A2
/mnt/data-weather/corr/A/032723A2
/mnt/data-weather/corr/A/032800A2
/mnt/data-weather/corr/A/032801A2
/mnt/data-weather/corr/A/032802A2
/mnt/data-weather/corr/A/032818A2
/mnt/data-weather/corr/A/032819A2
/mnt/data-weather/corr/A/032820A2
/mnt/data-weather/corr/A/032821A2
/mnt/data-weather/corr/A/032822A2
/mnt/data-weather/corr/A/032823A2
/mnt/data-weat

/mnt/data-weather/corr/A/021116A2
/mnt/data-weather/corr/A/021120A2
/mnt/data-weather/corr/A/021123A2
/mnt/data-weather/corr/A/021200A2
/mnt/data-weather/corr/A/021201A2
/mnt/data-weather/corr/A/021202A2
/mnt/data-weather/corr/A/021203A2
/mnt/data-weather/corr/A/021204A2
/mnt/data-weather/corr/A/021205A2
/mnt/data-weather/corr/A/021206A2
/mnt/data-weather/corr/A/021412A2
/mnt/data-weather/corr/A/021413A2
/mnt/data-weather/corr/A/021414A2
/mnt/data-weather/corr/A/021415A2
/mnt/data-weather/corr/A/021416A2
/mnt/data-weather/corr/A/021417A2
/mnt/data-weather/corr/A/021418A2
/mnt/data-weather/corr/A/021419A2
/mnt/data-weather/corr/A/021420A2
/mnt/data-weather/corr/A/021421A2
/mnt/data-weather/corr/A/021422A2
/mnt/data-weather/corr/A/021423A2
/mnt/data-weather/corr/A/021500A2
/mnt/data-weather/corr/A/022315A2
